In [9]:
from pyquil.quil import Program
from pyquil.quilatom import Parameter, quil_exp
from pyquil.quilbase import DefGate
from pyquil.gates import H, RX
import numpy as np

In [10]:
# Define the new gate from a matrix
theta = Parameter('theta')
rzz = np.array([
    [quil_exp(1j * theta), 0, 0, 0],
    [0, -1*quil_exp(1j * theta), 0, 0],
    [0, 0, -1*quil_exp(1j * theta), 0],
    [0, 0, 0, quil_exp(1j * theta)]
    ])

gate_definition = DefGate('RZZ', rzz, [theta])
RZZ = gate_definition.get_constructor()

def add_step(p, theta):
    p += RZZ(theta,[0, 1])
    p += RZZ(theta,[2, 3])
    p += RZZ(theta,[1, 2])
    p += RX(theta, 0)
    p += RX(theta, 1)   
    p += RX(theta, 2)
    p += RX(theta, 3)

# Create our program and use the new parametric gate
N = 30
p = Program()
p += H(0)
p += H(1)
p += H(2)
p += H(3)
for i in range(N):
    add_step(p, np.pi/2)

In [11]:
print(p)

DEFGATE RZZ(%theta):
    EXP(i*%theta), 0, 0, 0
    0, -1*EXP(i*%theta), 0, 0
    0, 0, -1*EXP(i*%theta), 0
    0, 0, 0, EXP(i*%theta)

H 0
RZZ(pi/2) 0 1

